###**BlackArbs LLC Independent Research**


####SPDR ETF Composite Sector Valuation [Saturday 4.11.2015]
------------

The following Ipython Notebook examines the **Implied Cost of Capital (ICC)** method of valuation for purposes of trade/portfolio positioning. The goal is to identify asymmetric investing opportunities due to incongruence between *'recent'* historical valuations and forward looking expectations of earnings growth. 

I will attempt to accomplish the goal by first examining composite returns based on ETF category groupings. Then I will compare the historical data vs the forward looking **ICC** estimate to see if we can find some disagreement. 

**_Please note there will be some category overlap as some of the groupings include international sector ETF's while other groupings contain regional and/or country ETF's._ 
____

First set up the environment, lists and other necessary metadata

In [ ]:
# ================================================================== #
# composite returns; vol; risk adjusted returns; correlation matrix, ICC analysis

from pprint import pprint as pp
import pandas as p
import numpy as np
import pandas.io.data as web
from pandas.tseries.offsets import *
import datetime as dt
import math
import seaborn as sns
sns.set_style('white')

import matplotlib.pyplot as plt

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

import cufflinks 

# ================================================================== #

date_today = dt.date.today()
month = 'APR-2015'

# ~~~ Market Cap ~~~ #
Broad_mkts = ['THRK','RSCO'] # Russell 3000, Russell Small Cap Completeness
Large_cap  = ['ONEK','SPY','SPYG','SPYV'] # Russell 1000, sp500 (growth, value)
Mid_cap    = ['MDY','MDYG', 'MDYV'] # sp400 mid (growth, value)
Small_cap  = ['TWOK','SLY','SLYG','SLYV'] # russ 2K, sp600, (growth, value)

# ~~~ International/Global Equities ~~~ #
Global = [
        'DGT', #  global dow
        'BIK', # sp BRIC 40 ETF
        'GMM', # sp emerging mkts 
        'EWX', # sp emerging mkts small caps
        'CWI', # msci acwi ex-US
        'GII', # global infrastructure
        'GNR', # global natural resources
        'DWX', # intl dividends
        'GWL', # sp developed world ex-US 
        'MDD', # intl mid cap (2B-5B USD)
        'GWX'  # intl small cap (<2B USD)
        ]

Asia   = ['JPP','JSC','GXC','GMF'] # japan, smallcap japan, china, emg asiapac
Europe = ['FEZ','GUR','RBL','FEU'] # euro stoxx 50, emg europe, russia, stoxx europe 50
Latam  = ['GML'] # emg latin america
Africa = ['GAF'] # emg mideast/africa

# ~~~ Real Assets ~~~ #
Real_assets = [ 'RWO', # global real estate
                'RWX', # intl real estate ex-US
                'RWR'  # US select REIT
                ]        

# ~~~ sectors and industries ETF's ~~~ #
Sector = [
          'XLY','XHB','IPD','XRT',                   # consumer discretionary
          'XLP','IPS',                               # consumer staples
          'XLE','IPW','XES','XOP',                   # energy
          'XLF','KBE','KCE','KIE','IPF','KRE',       # financials
          'XLV','XBI','XHE','XHS','IRY','XPH',       # healthcare
          'XLI','XAR','IPN','XTN',                   # industrial
          'XLB','IRV','XME',                         # materials
          'XLK','MTK','IPK','XSD','XSW',             # technology
          'IST','XTL',                               # telecom
          'IPU','XLU'                                # utilities
          ]
   
stock_list = [Broad_mkts, Large_cap, Mid_cap, Small_cap, Global, Asia, Europe, Latam, Africa, Real_assets, Sector]

# ~~~ Category structure ~~~ #
cat = {'Broad_Market'          :['THRK','RSCO'],
       'Large_Cap'             :['ONEK','SPY','SPYG','SPYV'],
       'Mid_Cap'               :['MDY','MDYG', 'MDYV'], 
       'Small_Cap'             :['TWOK','SLY','SLYG','SLYV'],
       'Global_Equity'         :['DGT','BIK','GMM','EWX','CWI','GII','GNR','DWX','GWL','MDD','GWX'],
       'AsiaPac_Equity'        :['JPP','JSC','GXC','GMF'],
       'Europe_Equity'         :['FEZ','GUR','RBL','FEU'],
       'Latam_MidEast_Africa'  :['GML','GAF'],
       'Real_Estate'           :['RWO','RWX','RWR'],
       'Consumer_Discretionary':['XLY','XHB','IPD','XRT'],
       'Consumer_Staples'      :['XLP','IPS'],                         
       'Energy'                :['XLE','IPW','XES','XOP'],                   
       'Financials'            :['XLF','KBE','KCE','KIE','IPF','KRE'],
       'Healthcare'            :['XLV','XBI','XHE','XHS','IRY','XPH'],
       'Industrial'            :['XLI','XAR','IPN','XTN'],
       'Materials'             :['XLB','IRV','XME'],
       'Technology'            :['XLK','MTK','IPK','XSD','XSW'],
       'Telecom'               :['IST','XTL'],                            
       'Utilities'             :['IPU','XLU']
        }    

filepath   = r'C:\Users\Owner\Documents\Visual Studio 2013\Projects\iVC_Reporting_Engine\PythonApplication2\\'

Now I define a convenience function to grab stock prices. 

In [ ]:
# ================================================================== #
# get prices
def get_px(stock, start, end):
    '''
    Function to call Pandas' Yahoo Finance API to get daily stock prices.
    
    Parameters:
    ==========
    stock = type('str'); stock symbol 
    start = 3 business days before today; datetime date_today object offset by pandas.DateOffset method 
    end   = today; datetime date_today object

    Returns:
    ========
    time series = Pandas.Series object corresponding to stock symbol, and start/end dates
    **Note that if price column is not specified the function will return a Pandas.DataFrame object
    '''      
    try:
        return web.DataReader(stock, 'yahoo', start, end)['Adj Close']
    except Exception as e:
        print( 'something is fucking up' )

px = p.DataFrame()
for category in stock_list:
    for stock in category:
        px[stock] = get_px( stock, date_today - 252 * BDay(), date_today )

Calculate the log returns for our ETF's then construct the dataframe with the proper multi-index for grouping

In [ ]:
# ================================================================== #
# construct dataframe and proper multi index
log_rets = np.log( px / px.shift(1) ).dropna()

lrets = log_rets.T.copy()
lrets.index.name = 'ETF'
lrets['Category'] = p.Series()

for cat_key, etf_val in cat.items():
    for val in etf_val:
        if val in lrets.index:
            idx_loc = lrets.index.get_loc(val)
            lrets.ix[idx_loc,'Category'] = cat_key
        else:
            pass

lrets.set_index('Category', append=True, inplace=True)
lrets = lrets.swaplevel('ETF','Category').sortlevel('Category')
lrets.head()

Calculate the cumulative returns of each ETF

In [ ]:
# ================================================================== #
# cumulative returns of ETF's
cum_rets = lrets.groupby(level='Category').cumsum(axis=1)
cum_rets.head()

Calculate a composite cumulative return based on the ETF categories and output the result. Each ETF is given equal weighting within its own category. 

In [ ]:
# ================================================================== #
# composite groupings of cumulative ETF returns (equally weighted intra-category mean returns)
composite_rets = p.DataFrame()
for label in cat.keys():
    composite_rets[label] = cum_rets.ix[label].mean(axis=0) # equal weighted mean
    
comp_rets = np.round(composite_rets.copy(),4) # rounding

# to demonstrate the enhanced interactivity of Ploty charts first plot matplotlib
%matplotlib inline

size=(10,8)
composite_rets.plot(figsize=size)
plt.tight_layout()


The basic plot is ok but how much can you really interpret from the previous time series given that there are 19 different categories? Probably not much. Good thing I have a solution. 

For this application I use a combination of **Plotly's API** and the **Cufflinks** module to create interactive charts. Plotly's charts allow for: zoom, pan, zoom-in, zoom-out, autoscale in addition to the awesome interactive data display. 

Below I create a convenince function to construct the layout I want to pass to the **df.iplot** method. 

In [ ]:
# ~~~~~ plot code ~~~~~ 
# function to create Plotly 'Layout' object

def create_layout( main_title, y_title ):
    '''
    Function to create custom Plotly layout object to pass to Cufflinks df.iplot() method
    
    Parameters:
    ==========
    
    main_title = type('str')
    y_title    = type('str')

    Returns:
    ========
    plotly_layout = Plotly Layout object basically constructed using a JSON or Dict structure    
    '''    
    plotly_layout = Layout(
        # ~~~~ construct main title
            title=main_title,
            font=Font(
                family='Open Sans, sans-serif',
                size=16,
                color='SteelBlue'
            ),
        # ~~~~ construct X axis
        xaxis=XAxis(
            title='$Date$',
            titlefont=Font(
                family='Open Sans, sans-serif',
                size=18,
                color='SteelBlue'
            ),
            showticklabels=True,
            tickangle=-30,
            tickfont=Font(
                family='Open Sans, sans-serif',
                size=11,
                color='black'
            ),
            exponentformat='e',
            showexponent='All'
        ),
        # ~~~~ construct Y axis
        yaxis=YAxis(
            title= y_title,
            titlefont=Font(
                family='Open Sans, sans-serif',
                size=18,
                color='SteelBlue'
            ),
            showticklabels=True,
            tickangle=0,
            tickfont=Font(
                family='Open Sans, sans-serif',
                size=11,
                color='black'
            ),
            exponentformat='e',
            showexponent='All'),
        # ~~~~ construct figure size
            autosize=False,
            width=750,
            height=500,
            margin=Margin(
            l=60,
            r=30,
            b=70,
            t=50,
            pad=5
            ),
        # ~~~~ construct legend      
            legend=Legend(
            y=0.5,
            #traceorder='reversed',
            font=Font(
                family='Open Sans, sans-serif',
                size=10,
                color='Black'
        ),            
        )
    )
    return plotly_layout

See the resulting plot below

In [ ]:
# test the function
title = '<b>Cumulative Log Returns of Composite ETF Sectors [1 Year]</b>'
y_label = '$Returns$'

custom_layout_1 = create_layout( title, y_label )
comp_rets.iplot(theme='solar',filename='{}_{}'.format(title, date_today), layout=custom_layout_1, world_readable=True)

Calculate the ETF category standard deviation of returns. Then construct a moving average of the composite sigmas and plot the data

In [ ]:
# ================================================================== #
# composite rolling std

sigmas = lrets.groupby(level='Category').std() # equal weighted std

composite_sigs = p.DataFrame()
for label in cat.keys():
    composite_sigs[label] = sigmas.ix[label] 

rsigs = p.rolling_mean( composite_sigs, window=60 ).dropna()*math.sqrt(60)

# ~~~~~ plot code
title = '<b>60-Day Moving Average of Standard Deviation</b>'
#y_label = r'$return \ \sigma$'
y_label = r'$\sigma \ of \ returns$'

custom_layout_2 = create_layout( title, y_label )
rsigs.iplot(theme='white',filename='{}_{}'.format(title, date_today), layout=custom_layout_2, world_readable=True)

Now I calculate a risk adjusted return metric and plot. For simplicity I assume there is no benchmark ETF return. I use the formula: 
##$$R_{risk} = \mu/\sigma$$

In [ ]:
# ================================================================== #
# composite rolling risk adjusted returns

mean_rets = lrets.groupby(level='Category').mean() # equal weighted mean
#risk_rets = (mean_rets-lrets.loc['Global_Equity','DGT'])/sigmas
#risk_rets = mean_rets/sigmas

composite_risk_rets = p.DataFrame()
for label in cat.keys():
    composite_risk_rets[label] = mean_rets.ix[label] 

rs = p.rolling_mean( composite_risk_rets, window=60 ).dropna() 
risk_rets = rs/rsigs

# ~~~~~ plot code
title = r'<b>60 day Moving Average of Composite Risk-Adjusted Returns</b>'
y_label = '$\mu/\sigma$$'

custom_layout_3 = create_layout( title, y_label )
risk_rets.iplot(theme='white', filename='{}_{}'.format(title, date_today), layout=custom_layout_3, world_readable=True)

For portfolio positioning/rebalancing we need to see the correlations of the return data. To do that I construct a composite correlation matrix and plot the results. 

In [ ]:
# ================================================================== #
# correlation matrix of composite ETF groups' risk adjusted returns
cor = risk_rets.corr()

# ~~ plot code
f, ax = plt.subplots(figsize=(11,11))

cmap = sns.diverging_palette(h_neg=12, h_pos=144, s=91, l=44, sep=29, n=12, center='light',as_cmap=True)
sns.corrplot(cor, annot=True, sig_stars=False, diag_names=False, cmap=cmap, ax=ax)
ax.set_title('Composite ETF Groups Correlation Matrix', fontsize=18)

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(13)

f.tight_layout()

Now that we have our historical return analysis I need to import my **ICC** estimates. 

#### A quick review of the **`Implied Cost of Capital (ICC)`** model for the uninitiated:
___

The **ICC** model is basically an **internal rate of return (IRR)** metric that uses the **Residual Income Model** of valuation. I present the basic formula below:

###$${Price}_{0} = {BVPS}_{0} +\sum\limits_{t=1}^{T} \frac {RI_{t}}{(1 + r)^t} +\frac {{Terminal} \ {Value}}{(1 + r)^t}$$

_where_: 
- $RI_{t} =$ ${EPS}_{t} - $$({r} * {BVPS}_{t-1})$
- ${Terminal}$ ${Value} =$ ${EPS}_{t} * (1 + g) / (r - g)$
- $g =$ $estimated$ $long$ $term$ $growth$ $rate$
- $r = $$cost$ $of$ $equity =$$cost$ $of$ $capital$ 

By solving for **r** aka the **cost of equity** we can estimate a market implied cost of equity or **ICC** for our purposes. This methodology allows us to solve for the market expected rate of growth over the estimated period that equates the future value of the stock to today's prices. The beauty of the metric is that it uses analysts' consensus **EPS** forecast to "solve backwards" to today's prices giving us a view into institutional investors' market expectations. 

Many investors feel that equity analysts' estimates are "bullshit" or consistently "wrong" but in this context analyst accuracy is largely irrelvant. The simple reason being that the market trades on expectations of future events not historical data. The thing is no one *knows* the future so looking at analyst estimates in hindsight is counterproductive. Remember the goal in calculating this metric is to take a snapshot of market expectations today *for the future*; literally the definition of **"forward looking"**. Ideally what the **ICC** captures is a measure of **"market sentiment"** or **"market psychology"**. Like most competitive games, if you can understand what your opponents' expectations are you have a decided advantage. 

***A simple example:*** If the calculation reveals the market expectations for future earnings growth of a sector like **`Utilities (XLU)`** is relatively low compared to its historical stock performance, an astute investor would interpret the signal as **`"the sector is ripe for correction and likely to surprise to the downside"`**. This could be because earnings estimates are declining or the stock price has already "priced in" future growth. The point being that it would be unwise to be a buyer at these hypothetical levels as the data hints at a likely "top" in value barring any opposing identifiable catalyst. 

In [ ]:
# ================================================================== #
# import ICC estimates
frame = p.read_csv( filepath+'Spdr_ICC_est_{}.csv'.format(date_today) , index_col=0 ).dropna()
# ================================================================== #
# group ICC data by category
f        = frame.copy()
grp      = f.groupby('Category')
grp_mean = grp.mean().sort('ETF_ICC_est', ascending=False)

In [ ]:
grp_mean = np.round( grp_mean['ETF_ICC_est'], 4 )
grp_mean

I'm a fan of z_scoring data like this so I can get a better sense of the extreme relative values 

In [ ]:
def z_score(df):
    return ( df - df.mean() ) / df.std()

#z_grp = (grp_mean - grp_mean.mean()) / grp_mean.std()
z_grp = z_score(grp_mean)

plt.figure()
z_grp.plot('barh', figsize=size, alpha=.8)
plt.axvline(0, color='k')
plt.title('Z Score of ICC Estimates by Category', fontsize=20)
plt.xlabel('$\sigma$', fontsize=20)
plt.ylabel('Category', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)


For comparative purposes I use the an average of the last 30 day's risk adjusted returns by ETF category. I then z score the data and plot it

In [ ]:
# last 30 days average category risk adjusted returns
date_mask = date_today - 30 * BDay()
l_30 = risk_rets.ix[date_mask:].mean().order(ascending=False) 
l_30

In [ ]:
# z scored and plotted
z_l_30 = z_score(l_30)

plt.figure()
z_l_30.plot('barh', figsize=size, color='r', alpha=.5)
plt.axvline(0, color='k')
plt.title('Z Score of Average Risk Adjusted Returns for last 30 days', fontsize=20)
plt.xlabel('$\sigma$', fontsize=20)
plt.ylabel('Category', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=14)

Finally I construct a dataframe comprised of both series of z scored data and plot them for comparison purposes. Again we are looking for 'significant' discrepencies between historical return data and market implied EPS growth expectations. Let's examine the results.

In [ ]:
z_data = p.DataFrame()
z_data['Z_ICC estimates'] = z_grp
z_data['Z_risk adj returns'] = z_l_30
# z_data.head()

cadet_blue = '#4e7496'

fig = plt.figure()
with p.plot_params.use('x_compat', True):
    z_data['Z_ICC estimates'].plot('barh', figsize=size, color=cadet_blue)
    z_data['Z_risk adj returns'].plot('barh',figsize=size, color='r', alpha=.5)
plt.axvline(0, color='k')
plt.title('Z Scores Comparison', fontsize=20)
plt.xlabel('$\sigma$', fontsize=24)
plt.ylabel('Category', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(loc='best', prop={'weight':'demibold','size':12})

There are some interesting results here. 

### Potential Long Positions:
___
 - **Europe_Equity** is intriguing. Already off to a relatively strong start on the year, they are actively devaluing the `EURO` and effectively using the `ECB` as put on the bond market. If the `ECB` can replicate what the `FED` did for U.S. risk assets European equity markets could be setting up for a long term bull run. At the very least we government tailwinds behind any long position. 
 - **Financials.** Comparatively they have the highest implied growth rate moving forward. Generally I'm not a fan of the sector due to the complexity and diversity of the firms within it but broadly speaking, financials have room to run if the market believes the `FED` will ever raises rates. It is assumed a rise in rates would improve the net interest margin for domestic banking institutions.
 - **Utilities** look like they could be a potential dip buy for a swing trade. The risk here is the built in headwind of a potential rate hike by the `FED`. Generally speaking utilities compete with other fixed-income asset classes for fund flows. If interest rates rise then there are more investable options for yield seeking investors. 
 
### Potential Short Positions:
___
 - **Mid Caps** look ripe for correction if i may quote my previous example. Market implied growth is in the bottom half of z scored data but a look at the risk adjusted returns relative to the other sectors shows massive outperformance. To me that would indicate an overbought situation with asymmetric risk to the downside. At the very least the mid cap grouping warrants more investigation for overvalued ETF's and individual stocks. If price action confirms my bear bias I would look to get short. 
 - **Small Caps** also look relatively overvalued. ICC estimates show low relative market implied growth yet the risk adjusted returns have outperformed all other categories except Mid Caps.
 - **Healthcare** is also interesting from an overvalued perspective. Clearly healthcare has the lowest market implied growth of the categories, but this is combined with solid outperformance. I lean towards a `neutral` bias here or a `no play` because there are structural macro shifts happening demographically. The population is aging/retiring faster than replacement. As a result there is a macro tailwind supporting the sector.
___

_I conclude this analysis with the disclaimer that these calculations are presented `"as is"` and the data was aggregated from several sources. I recommend doing your own due diligence before taking any investment action and to stay within your personal risk/return objectives._ 

_I expect to refine this model as necessary as I use this methodology to get a macro valuation perspective._ 


####**For comments, questions, and feedback contact me via:**
####email:   **`bcr@blackarbs.com`** 
####twitter: **`@blackarbsCEO`**
___

_**Data Sources:** Yahoo Finance, Marketwatch, SPDR ETFs_<br />_**Acknowledgements:** Ipython Notebook Styling modded from Plotly and Cam Davidson-Pilon custom CSS_

In [ ]:
from IPython.core.display import HTML
import requests
styles = requests.get("https://raw.githubusercontent.com/BlackArbsCEO/BlackArbsCEO.github.io/Equity-Analysis/Equity%20Analysis/custom.css")
HTML(styles.text)